In [1]:
#import nltk
#nltk.download('punkt')

In [2]:
# !pip install translate

In [3]:
#!pip install langdetect


In [4]:
import re
import pandas as pd
from pandas import DataFrame,Series
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import numpy as np
from collections import Counter
from translate import Translator
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [5]:
# Get Top10 Pizza restaurants based on the count of the restaurants 
# info includes number of restaurants and its corresponding category
pizzatop10=pd.read_csv("Pizzatop10count.csv",names=["number","name","info"])[["name","info"]][1:]
pizzatop10





,name,info
1,Pizza Hut,"[320, 'Restaurants, Italian, Pizza, Fast Food,..."
2,Domino's Pizza,"[209, 'Pizza, Caterers, Restaurants, Event Pla..."
3,Papa John's Pizza,"[152, 'Pizza, Restaurants', 'Tempe', 'AZ']"
4,Little Caesars Pizza,"[112, 'Restaurants, Pizza', 'Whitchurch-Stouff..."
5,Pizza Pizza,"[102, 'Pizza, Restaurants, Italian, Chicken Wi..."
6,Boston Pizza,"[69, 'Pubs, Pizza, American (Traditional), Nig..."
7,Papa Murphy's,"[68, 'Restaurants, Pizza', 'Phoenix', 'AZ']"
8,Marco's Pizza,"[52, 'Italian, Restaurants, Pizza, Chicken Win..."
9,Pizza Nova,"[52, 'Restaurants, Pizza', 'Richmond Hill', 'ON']"
10,Hungry Howie's Pizza,"[40, 'Restaurants, Pizza', 'Charlotte', 'NC']"


In [6]:
#All pizza restaurants' reviews
reviewall=pd.read_csv("pizza_review.csv")
len(reviewall['review_id'])


394428

In [7]:
#Get the corresponding top10 restaurants' review
reviewall['info']=reviewall['name'].map(pizzatop10.set_index('name')['info'])
reviewtop10=reviewall.loc[reviewall['info'].isin(list(pizzatop10['info']))]
reviewtop10[:3]  #Only show the first 3 lines



,Unnamed: 0,Unnamed: 0.1,business_id,cool,date,funny,review_id,stars,text,useful,user_id,name,info
18,267,267,fweCYi8FmbJXHCqLnwuk8w,0,2017-06-27 23:35:47,0.0,PnKm8y0thZ5ZdWhuOk7Opg,5.0,"Hands down, this is the best pizza place in Me...",0.0,VJvQiGrmtFXAmswRRIWYYw,Marco's Pizza,"[52, 'Italian, Restaurants, Pizza, Chicken Win..."
32,600,600,pif8Io-Jn2veckVGzkEyJw,0,2013-11-14 01:01:56,1.0,bLgMDEj1EXOuZ_94P5I7jA,4.0,I've never had a bad experience here.\r\r\n\r\...,0.0,renPzRDqMZpMaHiCD_e1_A,Pizza Hut,"[320, 'Restaurants, Italian, Pizza, Fast Food,..."
38,751,751,C2skZUb7BLpv3e78Mcf5cA,0,2018-09-03 01:40:55,0.0,5haHR9yvGIUQ-sFMrAzsQQ,1.0,We order from this Pizza Hut often and always ...,0.0,txHu-M3p2tLKSpNGMWUO0Q,Pizza Hut,"[320, 'Restaurants, Italian, Pizza, Fast Food,..."


In [8]:
len(reviewtop10)  #Total number of reviews for top 10 pizza chain restaurants


17951

In [9]:
#Reviewinfo includes the review text, the stars, the pizza restaurant name and the user_id of that review
reviewinfo=reviewtop10[["text","stars","name","user_id"]]
reviewinfo[:3]  #Only show the first 3 lines


,text,stars,name,user_id
18,"Hands down, this is the best pizza place in Me...",5.0,Marco's Pizza,VJvQiGrmtFXAmswRRIWYYw
32,I've never had a bad experience here.\r\r\n\r\...,4.0,Pizza Hut,renPzRDqMZpMaHiCD_e1_A
38,We order from this Pizza Hut often and always ...,1.0,Pizza Hut,txHu-M3p2tLKSpNGMWUO0Q


In [10]:
#Get only the text part
reviewtext=reviewtop10["text"]
reviewtext[:3]  #Only show the first 3 lines




18    Hands down, this is the best pizza place in Me...
32    I've never had a bad experience here.\r\r\n\r\...
38    We order from this Pizza Hut often and always ...
Name: text, dtype: object

# Translate different languages into English

In [11]:
language=[]
for review in reviewtext:
    language.append(detect(review))


In [13]:
set(language)  ##Language types

{'af', 'da', 'en', 'es', 'fr', 'hr', 'it', 'pt', 'sk', 'so', 'sw'}

In [14]:
## Do review translation
translatereview=[]
for review in reviewtext:
    languagetype=detect(review)
    if languagetype=='en':
        translatereview.append(review)
    else:
        if languagetype=='af':
            lang='Afrikaans'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='de':
            lang='German'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='da':
            lang='Danish'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='es':
            lang='spanish'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='et':
            lang='estonian'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='fr':
            lang='french'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='hr':
            lang='croatian'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='it':
            lang='italian'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='pl':
            lang='polish'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='pt':
            lang='portuguese'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='sk':
            lang='slovak'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='so':
            lang='somali'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        elif languagetype=='sw':
            lang='swahili'
            translator= Translator(from_lang=lang,to_lang="english")
            translation = translator.translate(review)
            translatereview.append(translation)
        else:
            translatereview.append(review)


In [15]:
dftranslate=DataFrame(translatereview)
dftranslate.to_csv('translate.csv')


In [16]:
transreview=pd.read_csv('translate.csv')["0"]
transreview[:3] #Show only the first three lines


0    Hands down, this is the best pizza place in Me...
1    I've never had a bad experience here.\r\r\n\r\...
2    We order from this Pizza Hut often and always ...
Name: 0, dtype: object

In [17]:
##Merge the translated review with the other information like stars, name and user_id
transreview1=DataFrame(transreview)
transreview1["stars"]=reviewinfo["stars"].values
transreview1["name"]=reviewinfo["name"].values
transreview1["user_id"]=reviewinfo["user_id"].values
transreview1[:3]


,0,stars,name,user_id
0,"Hands down, this is the best pizza place in Me...",5.0,Marco's Pizza,VJvQiGrmtFXAmswRRIWYYw
1,I've never had a bad experience here.\r\r\n\r\...,4.0,Pizza Hut,renPzRDqMZpMaHiCD_e1_A
2,We order from this Pizza Hut often and always ...,1.0,Pizza Hut,txHu-M3p2tLKSpNGMWUO0Q


In [18]:
transreview1.to_csv('translateinfo.csv')


In [92]:
transinfo=pd.read_csv('translateinfo.csv')[['0','stars','name','user_id']]
transinfo[:3] ##Show only first 3 lines as an example

,0,stars,name,user_id
0,"Hands down, this is the best pizza place in Me...",5.0,Marco's Pizza,VJvQiGrmtFXAmswRRIWYYw
1,I've never had a bad experience here.\r\r\n\r\...,4.0,Pizza Hut,renPzRDqMZpMaHiCD_e1_A
2,We order from this Pizza Hut often and always ...,1.0,Pizza Hut,txHu-M3p2tLKSpNGMWUO0Q


In [20]:
##Get userweight we computed from userweight.csv
userweight=pd.read_csv("userweight.csv")[['user_id','weight']]



In [21]:
#Merge the translated review with the corresponding user weight
transinfo['weight']=transinfo['user_id'].map(userweight.set_index('user_id')['weight'])



In [22]:
##Get the translated review text
reviewtext=transinfo['0']



# Change Review to sentences

In [23]:
#Change each review into sentences, finally we get a list of list, each list include the sentences of one review
def text_to_sentence(textdf):
    textlist=textdf.tolist()
    sentencelist=[]
    for review in textlist:
        sentences=sent_tokenize(review)
        sentencelist.append(sentences)
    return sentencelist




In [24]:
pizza10sent=text_to_sentence(transreview)
pizza10sent[0]  #Show only the first review after sentence breaking


['Hands down, this is the best pizza place in Mentor!',
 'Their pizza is so delicious that the smell alone is enough to make you drool!',
 'The cheesy bread is soft and cheesy and just goodness.',
 'The pizza is good for leftovers and the next day.',
 'The service and staff is very friendly.',
 '5 stars for sure!']

# Find the basic if suggestions raised by the customers for top10 pizza restaurants

In [25]:
#Find the if sentences, these sentences might be the condition and suggestion from the customer directly to Pizza Hut and customer.
#Eg: if you're going to a pizza hut in this area, i recommend this one
#Eg: if they were not up for delivering that late, a curiosity call would be great so i don't sit at home hungry.
def find_if_suggestion(text):
    suggestion=[]
    for review in text:
        for sentence in review:
            sentence=sentence.lower()
            if 'if ' in sentence or 'If ' in sentence or 'IF ' in sentence or 'iF ' in sentence:
                suggestion.append(sentence)
    return suggestion
               

In [26]:
pizza10_suggestion=find_if_suggestion(pizza10sent)
pizza10_suggestion[5:7] ##Show only 2 suggestions as example  





['if it was a busy night, this would all be more understandable but it was so dead in that restaurant.',
 'it took a while for our server to figure out if we can pay with a gift card and cash or not.']

In [27]:
len(pizza10_suggestion)


5753

In [28]:
#Get stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stopwords1=stop_words[:(stop_words.index('very')+1)] #these stopwords include pronowns and positive helping verbs.
stopwords1=stopwords1+["i've"]                



In [29]:
#First step  top 10 pizza data cleaning mainly deal with pronowns and helping verbs (stopwords1)
def pizza10clean1(text):
    pizza10clean=[]
    for review in text:
        s=[]
        for sentence in review:
            sentence = sentence.lower()
            if 'if' not in sentence:
                sentence = re.sub('not only (\w+?), but also', '\\1 and ', sentence) # change not only but also to and structure
                sentence=sentence.split(" ")
                sentence=[x for x in sentence if not x.strip() in stopwords1] #delete pronowns and helping verbs
                sentence=" ".join(sentence)
                s.append(sentence)
        pizza10clean.append(s)
    return pizza10clean



In [30]:
pizza10sent1=pizza10clean1(pizza10sent)



In [31]:
#Some other stopwords like frequency words and again the positive helping verbs
stopwords2=['here']+['always']+['usually']+['often']+['me']+['im']+['is']+['are']+['was']+['were']+['do']+['did']+['have']+['will']+['it']
print(stopwords2)


['here', 'always', 'usually', 'often', 'me', 'im', 'is', 'are', 'was', 'were', 'do', 'did', 'have', 'will', 'it']


In [32]:
#This function deal with negative words and connect them with the verbs behind it and then again delete those 
#positive helping verbs and some other useless frequency verbs
def pizza10clean2(text):
    pizza10clean=[]
    for review in text:
        s=[]
        for sentence in review:
            sentence = re.sub('n\'t\s', ' not', sentence)
            sentence = re.sub('not\s', ' not', sentence) #change not great to notgreat
            sentence = re.sub('never\s',' never',sentence) 
            sentence = re.sub('nothing\s',' nothing',sentence)
            sentence=sentence.translate(str.maketrans({key: None for key in string.punctuation})) #delete punctuation marks
            sentence=sentence.split(" ")
            sentence=[x for x in sentence if not x.strip() in stopwords2]
            for i in range (len(sentence)-1):
                if sentence[i].strip().startswith("not"):
                    sentence[i]=sentence[i]+sentence[i+1] #connect notgreat pizza into notgreatpizza
                    sentence[i+1]=""
            sentence=" ".join(sentence)
            s.append(sentence)
        pizza10clean.append(s)
    return pizza10clean



In [33]:
pizza10sent2=pizza10clean2(pizza10sent1)
pizza10sent2[0:2] ##Show first 2 reviews as example after data cleaning


[['hands down best pizza place mentor',
  'pizza delicious smell alone enough make drool',
  'cheesy bread soft cheesy just goodness',
  'pizza good leftovers next day',
  'service staff friendly',
  '5 stars sure'],
 [' neverbad experience',
  'delivery pickup whatever food turns good',
  'yesterday saving grace papa johns let severely',
  'pizza hut quoted hourish wait time which fine know promised 3040 minutes ends hour like pjs wasand house piping hot delicous pizza 25 minutes',
  'love']]

In [34]:
#Add stars, name ,user_id, userweight to the cleaned translated review text
pizza10info=[]
for i in range (len(pizza10sent2)):
    pizza10info.append((pizza10sent[i],transinfo['stars'][i],transinfo['name'][i],transinfo['user_id'][i],transinfo['weight'][i]))

pizza10info[0:2] ##Show first 2 as example


[(['Hands down, this is the best pizza place in Mentor!',
   'Their pizza is so delicious that the smell alone is enough to make you drool!',
   'The cheesy bread is soft and cheesy and just goodness.',
   'The pizza is good for leftovers and the next day.',
   'The service and staff is very friendly.',
   '5 stars for sure!'],
  5.0,
  "Marco's Pizza",
  'VJvQiGrmtFXAmswRRIWYYw',
  0.000275388635426039),
 (["I've never had a bad experience here.",
   'delivery, pickup, whatever, the food always turns out good, for me.',
   "Yesterday they were my saving grace, when Papa John's let me down SEVERELY.",
   "Pizza Hut quoted me an hour-ish wait time (which is fine when I know about it, not when i'm promised 30-40 minutes and it ends up being over an hour like PJ's was)...and they were there at my house with my piping hot, delicous pizza in about 25 minutes.",
   'LOVE.'],
  4.0,
  'Pizza Hut',
  'renPzRDqMZpMaHiCD_e1_A',
  0.10796669498067801)]

# Extract nown vocabulary for top 10 pizza restaurants

In [35]:
nown_list = ['NN','NNS','NNPS','NNP']   #https://blog.csdn.net/john159151/article/details/50255101

In [36]:
def find_nown_vocabulary(text):
    vocabularylist = []
    for review in text:
        for sentence in review:
            tag = nltk.pos_tag(word_tokenize(sentence))
            for i in range(len(tag)):
                if tag[i][1] in nown_list:
                    vocabularylist.append(tag[i][0])
    return vocabularylist


In [37]:
nown_vocabulary = find_nown_vocabulary(pizza10sent2)
nown_vocabulary[:10]



['hands',
 'place',
 'mentor',
 'smell',
 'drool',
 'cheesy',
 'bread',
 'cheesy',
 'goodness',
 'leftovers']

In [38]:
len(nown_vocabulary) 

325318

In [93]:
vocabularynown = np.array(nown_vocabulary)
nown_vocabulary_dict=Counter(vocabularynown)  # {label:sum(label)}
nown_vocabulary_dict.most_common(5)


[('pizza', 24165),
 ('order', 11138),
 ('time', 7000),
 ('service', 5603),
 ('delivery', 4873)]

# Extract adj vocabulary for top 10 Pizza restaurants

In [40]:
adj_tag = ['JJ','JJR','JJS','RB','RBR','RBS'] 


In [41]:
def find_adj_vocabulary(text):
    vocabularylist = []
    for review in text:
        for sentence in review:
            tag = nltk.pos_tag(word_tokenize(sentence))
            for i in range(len(tag)):
                if tag[i][1] in adj_tag:
                    vocabularylist.append(tag[i][0])
    return vocabularylist

In [42]:
adj_vocabulary = find_adj_vocabulary(pizza10sent2)
#adj_vocabulary


In [96]:
adj_vocabulary=[x for x in adj_vocabulary if not x in ['pizza','just']]

In [94]:
vocabularyadj = np.array(adj_vocabulary)
adj_vocabulary_dict=Counter(vocabularyadj)  # {label:sum(label)}
adj_vocabulary_dict.most_common(5)   ##TOP 5 most common adjs



[('good', 5068),
 ('great', 3173),
 ('back', 3087),
 ('really', 2502),
 ('even', 2310)]

# Extract not vocabularies for top 10 Pizza restaurauts

In [45]:
def find_not_vocabulary(text):
    vocabulary=[]
    for review in text:
        for sentence in review:
            sentence=sentence.split(" ")
            sentence=[x for x in sentence if x.startswith("not") or x.startswith("never")]
            if sentence!=[]:
                vocabulary.append(sentence[0])
    return vocabulary



In [46]:
not_vocabulary = find_not_vocabulary(pizza10sent2)
not_vocabulary=[x for x in not_vocabulary if not x in ['not']]


In [95]:
vocabularynot = np.array(not_vocabulary)
not_vocabulary_dict=Counter(vocabularynot)  # {label:sum(label)}
not_vocabulary_dict.most_common(5)  ##Top 5 most common not phrases

[('neverorder', 361),
 ('neveragain', 170),
 ('nevergo', 166),
 ('nothing', 123),
 ('neverever', 92)]

# Adjective list (Combine the top 50 "adjective" list with top 50 "not " list)

In [48]:
not1=[x[0] for x in not_vocabulary_dict.most_common(50)]
adj1=[x[0] for x in adj_vocabulary_dict.most_common(50)]
adj_list=adj1+not1
adj_list[:3] ## Show only the first 3 for example

['good', 'great', 'back']

# Gain the feature matrix X and labels y

In [49]:
y=[x[1] for x in pizza10info]

In [50]:
binary_y=DataFrame(y)[0].apply(lambda x:1 if x>3 else 0)

In [51]:
user_weight=[x[4] for x in pizza10info]

In [52]:
# This is the word count matrix(17951*100), row represents each review and column represents each adjective words
N=len(pizza10sent2);p=100
X=np.zeros([N,p])
for i in range(N):
    for j in range (p):
        for sentence in pizza10sent2[i]:
            if sentence.find(adj_list[j])!=-1:
                X[i][j]+=1
            else:
                continue
X            
X[1] ##Take the second row as an example, this line represents the 100 common adjected words' count frequency


array([1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 3., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

#  Logistic regression (without userweight adjustment)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, binary_y, test_size=0.2, shuffle=True, random_state=123)

In [54]:
modellr_train = LogisticRegression(random_state=123, solver='sag')
modellr_train.fit(X_train,y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=123, solver='sag', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
modellr_train.coef_  #coefficients of linear regression model

array([[ 0.50311192,  1.29239829, -0.37381768,  0.15336853, -0.32713115,
        -0.32223852, -0.44635371,  0.19622809, -0.1765264 ,  0.0838625 ,
        -0.06273982,  0.10420632,  0.11398552, -0.48470866, -0.31640739,
        -0.18816979, -0.17308946,  1.32511646, -0.52609842, -0.7267471 ,
        -0.03048523,  1.11982061, -0.21740408,  0.09373896,  0.62442384,
         0.1941665 ,  0.13740169,  0.11841349, -2.93839308, -0.74394121,
        -0.10092547,  1.05161534, -0.26477731, -0.15920014, -0.17302249,
         0.03915269,  0.91422732, -1.82559428,  0.80205197,  2.02912966,
        -0.33712659, -0.71305232, -0.71516834,  0.18259643,  0.16445248,
         0.08860237, -0.13589944,  0.0249552 , -2.01216548, -0.94652905,
        -0.93123443, -1.69293125, -0.76270352, -0.82389617, -1.63849199,
         0.7413609 , -1.01317019,  1.43728708, -0.62028797,  1.49102661,
        -0.43683603, -0.19873996, -1.06526174, -0.54102271, -0.56539215,
        -0.7537161 , -0.98971899, -0.93814303, -1.3

In [56]:
# Check the logistic model accuracy without user weight adjustment
y_pred = modellr_train.predict(X_test)
num_correct_predictions = (y_pred == y_test).sum()
num_correct_predictions
accuracy=(num_correct_predictions / len(y_test)) * 100
accuracy

80.25619604566972

The model accuracy is above 80%, which is ok.

# Linear Regression without considering user weight

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=123)

In [58]:
linreg = LinearRegression(fit_intercept=False)
modellm_train=linreg.fit(X_train, y_train)
print (modellm_train)
print (linreg.coef_)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)
[ 0.80585011  1.36182955  0.01585113  0.23867353  0.027046    0.01989688
  0.0094359   0.3795964   0.01545671  0.10245094  0.14042614  0.27070849
  0.074725    0.04186619  0.20420194  0.3308061   0.13182834  1.5559086
  0.06573307 -0.00844682  0.11194231  1.32953576 -0.01703627  0.1703246
  0.82742065  0.11617336  0.20494038  0.20555855  0.01320586  0.04320024
  0.17766274  0.80708317  0.06512985 -0.25275775  0.09856874  0.12440467
  1.1752582   0.10848658  0.74476522  1.43582896  0.18905354  0.00620309
 -0.19623573 -0.1066453   0.38185567  0.1048467  -0.03256115  0.17797754
  0.10548872 -0.1580932   0.12747072 -0.07510487  0.01448535  0.04946933
 -0.05403813  0.28310318 -0.09325666  1.26894146  0.18135272  0.91371789
 -0.09574741 -0.09349543 -0.09025443  0.19521358 -0.2561834   0.09814911
 -0.2349563   0.03105837 -0.04110311 -0.18677994  0.60584188  0.06216545
  0.0311827   0.03501543 -0.22904457  1.48747923

In [59]:
#Check the MSE of this model
y_pred = modellm_train.predict(X_test)
MSElm=sum((y_pred-y_test)**2)/len(y_test)
MSElm

3.9343057852506362

The MSE of linear regression model is 3.934.

# Use full dataset and get the weight(coefficients) of each adjective words without user weight adjustment

In [60]:
linreg = LinearRegression(fit_intercept=False)
modellm=linreg.fit(X, y)

In [61]:
lm={"adj":adj_list,"lin_reg_weight":linreg.coef_}
weight=DataFrame(data=lm)
weight_lmsort=weight.sort_values(by="lin_reg_weight",ascending=False)
weight_lmsort.to_csv("weight.csv")


# Linear regression considering user weight (LWLR-local weighted linear regression)

In [62]:
w=1+np.array(user_weight)  #The weight vector is the userweight of each review plus 1


In [63]:
X_train, X_test, y_train, y_test,w_train,w_test = train_test_split(X, y, w,test_size=0.2, shuffle=True, random_state=375)

In [64]:
W_train=np.diag(w_train)  #Use the weight from the training set as the diagonal of the training weight matrix
W_test=np.diag(w_test)   #Use the weight from the test set as the diagonal of the test weight matrix




Following coefficients of LWLR model is generated from this formula $$\hat \beta=(X'WX)^{-1}(X'WY)$$

In [65]:
A=(X_train).T.dot(W_train).dot(X_train)  
A_inv=np.linalg.inv(A)  # inverse of (X'WX)
B=X_train.T.dot(W_train).dot(y_train) #calculate (X'WY)
lin_reg_weight_adj_train=A_inv.dot(B)  # caluculate (X'WX)^{-1}(X'WY)
lin_reg_weight_adj_train 



array([ 0.80903942,  1.31883608,  0.03958767,  0.2834962 ,  0.04468838,
       -0.01523993,  0.0128836 ,  0.42319252,  0.03479355,  0.13298831,
        0.15438527,  0.23418133,  0.07660867, -0.0059665 ,  0.17401661,
        0.30556473,  0.06817896,  1.51819295,  0.02173437, -0.01729857,
        0.15200634,  1.21586978,  0.09980095,  0.19605558,  0.84379385,
        0.10424209,  0.16924128,  0.20119745, -0.01797771,  0.06708151,
        0.1214455 ,  0.77729401,  0.02791339, -0.35495408,  0.18797074,
        0.1468556 ,  1.07123741,  0.13107984,  0.72803473,  1.45598882,
        0.22832028,  0.05443304, -0.15675609, -0.07918427,  0.36152144,
        0.16980515, -0.03413417,  0.08284363,  0.18658911, -0.14067613,
        0.02233459, -0.04383697, -0.03800959, -0.01643646,  0.05336574,
        0.31364153, -0.12541655,  1.37778809,  0.07177528,  0.89055952,
        0.32465264, -0.00544184, -0.10784624,  0.24601687, -0.2517763 ,
       -0.13517404, -0.13758196,  0.13245122, -0.02481823, -0.15

In [66]:
#Use the above model and fit into the test set and get the MSE of the LWLR model
ytestfit=[0]*len(y_test)  
for j in range (100):
    ytestfit+=X_test[:,j]*lin_reg_weight_adj_train[j]
MSElmadj=sum((ytestfit-y_test)**2)/len(y_test)
MSElmadj


3.682711383190153

From above 2 models the LWLR has smaller MSE than original linear regression, although the train test split has randomness, we still see some improvement by the user weight adjustment.

# Finally train the linear regression model adjusted by the user weight with full dataset

In [69]:
linreg = LinearRegression(fit_intercept=False)
modellm=linreg.fit(X, y)

In [70]:
adjlr={"adj":adj_list,"lin_reg_weight_adj":modellm.coef_}
adjweight=DataFrame(data=adjlr)
#adjweight.to_csv("adjsweight.csv")



# Choosing good and bad adjected words that has great weight(influence to the stars)

#  (1) Linear regression model

In [71]:
adjweight_lmsort=adjweight.sort_values(by="lin_reg_weight_adj",ascending=False)
adjweight_lmsort.to_csv("adjsweight.csv")


In [72]:
#Get positive adjected words from LWLR model with the threshold 1. (Only choose those words with coeff>1)
adjweight_lmsort=adjweight.sort_values(by="lin_reg_weight_adj",ascending=False)
adjweight_lmsortgood=adjweight_lmsort[adjweight_lmsort["lin_reg_weight_adj"]>1]
goodlm_adj=list(adjweight_lmsortgood["adj"].values)
goodlm_adj



['friendly',
 'neverdisappointed',
 'delicious',
 'notbeat',
 'great',
 'best',
 'neverproblem',
 'fast']

In [73]:
#Get negative adjected words from LWLR model with the threshold -0.25. (Only choose those words with coeff<-0.25)
adjweight_lmsortbad=adjweight_lmsort[adjweight_lmsort["lin_reg_weight_adj"]<-0.25]
badlm_adj=adjweight_lmsortbad["adj"].values
badlm_adj


array(['notgoingback', 'nevercome', 'notready', 'neverordered', 'later',
       'neverreally'], dtype=object)

In [74]:
#Combine the selected postive and negative adjective words and gain the final adjective list
final_adj_list=list(goodlm_adj)+list(badlm_adj)

# Find the corresponding nouns that are adjcent to the adjected words

In [75]:
##This function help choose sentences that contain the adjective words from the final adjective list.
def find_adj_sentence(text):
    list1=[]
    for i in range (len(text)):
        list2 = []
        for sentence in text[i]:
            sentence=sentence.split()
            for word in sentence:
                if word in final_adj_list:
                    list2.append(" ".join(sentence))
        list1.append(list2)
    return list1



In [76]:
pizza10sent3=find_adj_sentence(pizza10sent2)
pizza10sent3[:5] ##Show only 5 reviews after implementing the function


[['hands down best pizza place mentor',
  'pizza delicious smell alone enough make drool',
  'service staff friendly'],
 [],
 [],
 [],
 ['way owner handled situation great']]

In [77]:
#Find the corresponding nown that close to the selected adjective words in our final adjective list, make them into
#adjective-nown or nown-adjective pair and also get the name of the restaurant that this pair comes from.
adj_nownlist=[]
for j in range (len(pizza10sent3)):
    if pizza10sent3[j]==[]:
        pass
    else:
        for sentence in pizza10sent3[j]:
            tag=nltk.pos_tag(word_tokenize(sentence))
            for i in range(len(tag)):
                if tag[i][0] in final_adj_list:
                    try:
                        if tag[i+1][1] in nown_list:
                            adj_nownlist.append((tag[i][0],tag[i+1][0],pizza10info[j][2])) 
                    except:
                        if tag[i-1][1] in nown_list:
                            adj_nownlist.append((tag[i-1][0],tag[i][0],pizza10info[j][2]))
                        else:
                            continue

In [78]:
adj_nownlist[:5]  ##Show first 5 as example

[('delicious', 'smell', "Marco's Pizza"),
 ('staff', 'friendly', "Marco's Pizza"),
 ('situation', 'great', "Hungry Howie's Pizza"),
 ('best', 'part', 'Little Caesars Pizza'),
 ('great', 'pizza', "Hungry Howie's Pizza")]

# The noun and selected adjected  words pair for pizzahut restaurant

In [79]:
##Only choose those pairs from pizzahut restaurant and make it into a dictionary where the keys
## are the adjective words and the values are also dictionarys with counts of each nouns that close to the 
## adjective words
final_adj_nown_dict_pizzahut={}
for pair in adj_nownlist:
    if pair[2]=="Pizza Hut":
        if pair[0] in final_adj_list:
            if pair[0] not in final_adj_nown_dict_pizzahut:
                final_adj_nown_dict_pizzahut[pair[0]]={pair[1]:1}
            else:
                if pair[1] not in final_adj_nown_dict_pizzahut[pair[0]]:
                    final_adj_nown_dict_pizzahut[pair[0]][pair[1]]=1
                else:
                    final_adj_nown_dict_pizzahut[pair[0]][pair[1]]+=1
        elif pair[1] in final_adj_list:
            if pair[1] not in final_adj_nown_dict_pizzahut:
                final_adj_nown_dict_pizzahut[pair[1]]={pair[0]:1}
            else:
                if pair[0] not in final_adj_nown_dict_pizzahut[pair[1]]:
                    final_adj_nown_dict_pizzahut[pair[1]][pair[0]]=1
                else:
                    final_adj_nown_dict_pizzahut[pair[1]][pair[0]]+=1
        
        
        

In [80]:
final_adj_nown_dict__pizzahut_sort={}
for key in final_adj_nown_dict_pizzahut:
    final_adj_nown_dict__pizzahut_sort[key]=sorted(final_adj_nown_dict_pizzahut[key].items(),key=lambda x:x[1],reverse=True)



In [89]:
final_adj_nown_dict__pizzahut_sort["delicious"][:3]  ##Show only the first 3 nouns near to adj 'delicious' as an example


[('pizza', 15), ('staff', 6), ('food', 5)]

In [82]:
total1=0
for i in range (len(list(final_adj_nown_dict__pizzahut_sort.values()))):
    pizzahutnown_list=[x[1] for x in list(final_adj_nown_dict__pizzahut_sort.values())[i]]
    total1+=sum(pizzahutnown_list)

total1   ##Total number of nouns for Pizza Hut


1267

# The noun and selected adjective word pairs for the other 9 top pizza restaurants

In [83]:
final_adj_nown_dict_other9pizza={}
for pair in adj_nownlist:
    if pair[2]!="Pizza Hut":
        if pair[0] in final_adj_list:
            if pair[0] not in final_adj_nown_dict_other9pizza:
                final_adj_nown_dict_other9pizza[pair[0]]={pair[1]:1}
            else:
                if pair[1] not in final_adj_nown_dict_other9pizza[pair[0]]:
                    final_adj_nown_dict_other9pizza[pair[0]][pair[1]]=1
                else:
                    final_adj_nown_dict_other9pizza[pair[0]][pair[1]]+=1
        elif pair[1] in final_adj_list:
            if pair[1] not in final_adj_nown_dict_other9pizza:
                final_adj_nown_dict_other9pizza[pair[1]]={pair[0]:1}
            else:
                if pair[0] not in final_adj_nown_dict_other9pizza[pair[1]]:
                    final_adj_nown_dict_other9pizza[pair[1]][pair[0]]=1
                else:
                    final_adj_nown_dict_other9pizza[pair[1]][pair[0]]+=1
        
        
        

In [84]:
final_adj_nown_dict__other9pizza_sort={}
for key in final_adj_nown_dict_other9pizza:
    final_adj_nown_dict__other9pizza_sort[key]=sorted(final_adj_nown_dict_other9pizza[key].items(),key=lambda x:x[1],reverse=True)




In [88]:
final_adj_nown_dict__other9pizza_sort["fast"][:3]  ##Show only the first 3 nouns near to adj 'fast' as an example


[('food', 166), ('delivery', 130), ('service', 70)]

In [86]:
total2=0
for i in range (len(list(final_adj_nown_dict__other9pizza_sort.values()))):
    othernown_list=[x[1] for x in list(final_adj_nown_dict__other9pizza_sort.values())[i]]
    total2+=sum(othernown_list)

total2   ## Total number of nouns for other 9 pizza restaurants



5335